In [ ]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer

In [ ]:
data = pd.read_csv("Datasets/Amazon Customer Behavior Survey.csv")
print(data.head())

In [ ]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
# Convert the datetime object to a float value (timestamp)
data['Timestamp'] = data['Timestamp'].apply(lambda x: x.timestamp())

In [ ]:
data['Timestamp'].info

In [ ]:
data['Gender'].info

In [ ]:
# Create a mapping of gender categories to numeric values
gender_mapping = {
    'Female': 0,
    'Male': 1,
    'Prefer not to say': 2,
    'Other': 3
    # Add more categories and numeric values as needed
}

# Use a lambda function and map to convert the values to floats
data['Gender'] = data['Gender'].map(lambda x: gender_mapping.get(x, float('nan')))

In [ ]:
data['Gender'].value_counts()

In [ ]:
# Fit the imputer on the data and transform the Series
data['Gender'] = imputer.fit_transform(data['Gender'].values.reshape(-1, 1))

In [ ]:
data['Gender'].info

In [ ]:
data['Purchase_Frequency'].value_counts()

In [ ]:
pf_mapping = {
    'Few time a month': 0,
    'Less than once a month': 1,
    'Once a week': 2,
    'Once a month': 3,
    'Multiple times a week': 4
}

data['Purchase_Frequency'] = data['Purchase_Frequency'].map(lambda x: pf_mapping.get(x, float('nan')))

In [ ]:
data['Purchase_Frequency'].info

In [ ]:
# Create an instance of SimpleImputer with the desired strategy
imputer = SimpleImputer(strategy='mean')

In [ ]:
# Fit the imputer on the data and transform the Series
data['Purchase_Frequency'] = imputer.fit_transform(data['Purchase_Frequency'].values.reshape(-1, 1))

In [ ]:
# Convert the result back to a Pandas Series
data['Purchase_Frequency'].info

In [ ]:
data['Purchase_Categories'].value_counts()

In [ ]:
pc_mapping = {
    'Beauty and Personal Care': 0,
    'Clothing and Fashion': 1,
    'Beauty and Personal Care;Clothing and Fashion': 3,
    'Beauty and Personal Care;Clothing and Fashion;Home and Kitchen': 4,
    'Groceries and Gourmet Food;Beauty and Personal Care;Clothing and Fashion;Home and Kitchen;others': 5,
    'Clothing and Fashion;Home and Kitchen': 6,
    'Home and Kitchen': 7,
    'Beauty and Personal Care;Home and Kitchen': 8,
    'Clothing and Fashion;Home and Kitchen;others': 9,
    'Clothing and Fashion;others': 10,
    'Groceries and Gourmet Food': 11,
    'Groceries and Gourmet Food;Beauty and Personal Care;Clothing and Fashion;Home and Kitchen': 12,
    'Beauty and Personal Care;Clothing and Fashion;others': 13,
    'Groceries and Gourmet Food;Beauty and Personal Care;Clothing and Fashion': 16,
    'Home and Kitchen;others': 17,
    'Beauty and Personal Care;Clothing and Fashion;Home and Kitchen;others': 18,
    'Beauty and Personal Care;others': 19,
    'Groceries and Gourmet Food;Beauty and Personal Care': 20,
    'Groceries and Gourmet Food;Home and Kitchen;others': 21,
    'Groceries and Gourmet Food;Clothing and Fashion': 22,
    'Groceries and Gourmet Food;Home and Kitchen': 23,
    'Beauty and Personal Care;Home and Kitchen;others': 24,
    'Groceries and Gourmet Food;Beauty and Personal Care;Home and Kitchen': 25,
    'Groceries and Gourmet Food;Clothing and Fashion;Home and Kitchen': 26,
    'Groceries and Gourmet Food;Clothing and Fashion;Home and Kitchen;others': 27,
    'Groceries and Gourmet Food;Beauty and Personal Care;others': 28,
    'Groceries and Gourmet Food;Clothing and Fashion;others': 29,
    'Groceries and Gourmet Food;Beauty and Personal Care;Clothing and Fashion;others': 30}

data['Purchase_Categories'] = data['Purchase_Categories'].map(lambda x: pc_mapping.get(x, float()))

In [ ]:
data['Purchase_Categories'].value_counts()

In [ ]:
data['Personalized_Recommendation_Frequency'].value_counts()

In [ ]:
prf_mapping = {
    'No': 0,
    'Sometimes': 1,
    'Yes': 2
}

data['Personalized_Recommendation_Frequency'] = data['Personalized_Recommendation_Frequency'].map(lambda x: prf_mapping.get(x, float()))

In [ ]:
data['Personalized_Recommendation_Frequency'].value_counts()

In [ ]:
data['Personalized_Recommendation_Frequency'].info

In [ ]:
data.columns

In [ ]:
X = data[['Timestamp', 'age', 'Purchase_Frequency', 'Purchase_Categories', 'Personalized_Recommendation_Frequency', 'Shopping_Satisfaction']]

y = data['Gender']

In [ ]:
X.shape

In [ ]:
y = y.astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.5,shuffle=True, random_state=5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rf = RandomForestClassifier(n_estimators=100,max_features='sqrt')
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test,y_pred_rf,target_names=['Female', 'Male','Prefer not to say']))

score = metrics.accuracy_score(y_test, y_pred_rf)
print("accuracy:   %0.3f" % score)

In [ ]:
lgb = LGBMClassifier(objective='multiclass',boosting_type= 'gbdt',n_jobs = 5, 
          silent = True, random_state=5)
LGB_C = lgb.fit(X_train, y_train)


y_pred_lgb = LGB_C.predict(X_test)
print(classification_report(y_test,y_pred_lgb,target_names=['Female', 'Male','Prefer not to say']))

score = metrics.accuracy_score(y_test, y_pred_lgb)
print("accuracy:   %0.3f" % score)

In [ ]:
xgb_c = xgb.XGBClassifier(n_estimators= 100)
xgb_c.fit(X_train,y_train)
y_pred_x = xgb_c.predict(X_test)
print(classification_report(y_test,y_pred_x,target_names=['Female', 'Male','Prefer not to say']))


score = metrics.accuracy_score(y_test, y_pred_x)
print("accuracy:   %0.3f" % score)

In [ ]:
def predict_purchase_frequency(model, data, gender_type):
    # Prepare the input data for prediction
    input_data = X_test
    
    # Use the trained model to make predictions
    prediction = model.predict(input_data[['Timestamp', 'age', 'Purchase_Frequency', 'Purchase_Categories', 'Personalized_Recommendation_Frequency', 
                                           'Shopping_Satisfaction']])
    
    return prediction[0]  # Return the predicted purchase frequency

# Test the function with different gender types
gender_types = ['Female', 'Male', 'Prefer not to say']
for gender_type in gender_types:
    predicted_frequency = predict_purchase_frequency(xgb_c, data, gender_type)
    if predicted_frequency == 0:
        predicted_frequency = "Few times a month"
    elif predicted_frequency == 1:
        predicted_frequency = "Less than once a month"
    elif predicted_frequency == 2:
        predicted_frequency = "Once a week"
    elif predicted_frequency == 3:
        predicted_frequency = "Once a month"
    else:
        predicted_frequency = "Multiple times a week"
    print(f"Predicted Purchase Frequency for {gender_type}: {predicted_frequency}")